In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# 6.1.4 Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])
    

6.71231743781199 -52.112001758865915
-62.72887324206349 34.87611424812195
51.370215092309365 78.23651783936629
81.47929998847462 5.10927720995025
-19.30456262859755 0.4720531968677335
74.24871789892134 -157.14214718391023
63.01984419746998 -135.11686123036537
48.32432170831285 117.15040849241905
-76.31945453051365 -26.929272346833358
-51.69213041634276 -178.67855137146086
-76.01273279596055 151.15112997415923
37.78595587573599 112.17783449955596
-28.393724499502326 -53.46838894736152
29.454874064693755 86.46306537516341
4.271122546079553 -26.154577082239513
-20.133593185989014 133.58441870516646
32.77607103438936 -165.63818768336904
86.11832036486669 126.74953297530834
32.41909126497116 -81.10768036568396
76.91131047325206 -114.90141593705282
-46.132245154587096 146.06571264411616
-3.632914224975778 -92.16280344519303
-57.59795682688687 -83.43667786763494
-80.71048683550089 39.411223255485424
4.2874886654877855 113.9922422284788
-30.707324984484693 14.135372920047644
-27.4518248392084 

46.99367492743693 -172.4236569458894
33.28694694599615 -40.09294414784392
-21.923831069932206 96.3951444717382
-53.48268814083447 22.45352480394598
-78.41411533907944 23.05322434090212
19.33574843959201 29.694345745273125
73.53954752867062 25.932512404276565
-86.00003896390209 -30.552020995876376
-67.82328498618001 -150.69612189713385
53.415128459635696 -145.41507819276427
87.26451059270968 -117.91653195072394
80.33300734411634 108.42448259341086
-84.30205121495466 120.25739933281801
36.25282554200071 -134.44943597109796
-21.094224660198165 -135.03945748322622
61.727380789981254 -102.5192900288074
-64.30297796114935 21.56723373192409
69.34573779758045 138.3047869657981
-63.796878475342446 138.86843870200232
-87.99215493432617 147.8514439738534
-85.03727599360732 102.8145803488054
-76.78263649104359 -174.96287032296627
28.871157012357045 117.31298132378214
-73.79097879758844 165.47110849261225
5.0012453386170534 154.83272689434727
33.503472079543144 -129.68978920198893
-25.1358004982128

In [7]:
# 6.1.5 Generate Random World Cities
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

593